In [1]:
# Dependencies
import pandas as pd

In [2]:
# Load in file
student_file = "../Resources/students_complete.csv"
schools_file = "../Resources/schools_complete.csv"

In [3]:
# Open the students file in CSV
student_df = pd.read_csv(student_file)
schools_df = pd.read_csv(schools_file)

In [4]:
# Count total number of schools in district
total_schools = len(student_df["school_name"].unique())
total_schools

15

In [5]:
# Count total number of students
student_count = len(student_df)
student_count

39170

In [6]:
# Sum total budget
total_budget = schools_df["budget"].sum()
total_budget

24649428

In [7]:
# Calculate average math and reading scores
average_math = student_df["math_score"].mean()
average_reading = student_df["reading_score"].mean()

In [8]:
# Calculate number of of students passing for math and reading    
math_passing_df = student_df.loc[student_df["math_score"] >= 70]
count_math = len(math_passing_df)
reading_passing_df = student_df.loc[student_df["reading_score"] >= 70]
count_reading = len(reading_passing_df)

# Divide passing scores for math and reading by total student count
percentage_passing_math = count_math/student_count
percentage_passing_reading = count_reading/student_count

In [153]:
overall = student_df[(student_df["math_score"] >= 70) & (student_df["reading_score"] >= 70)].count()["Student ID"]
overall_passing = overall / student_count

In [154]:
# District Summary
district_df = pd.DataFrame({
    "Total Schools":[total_schools],
    "Total Students":f'{student_count:,}',
    "Total Budget":f'${total_budget:,}',
    "Average Math Score":[average_math],
    "Average Reading Score":[average_reading],
    "% Passing Math":f'{percentage_passing_math:.2%}',
    "% Passing Reading":f'{percentage_passing_reading:.2%}',
    "% Overall Passing":f'{overall_passing:.2%}'
})
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428",78.985371,81.87784,74.98%,85.81%,65.17%


In [155]:
school_summary_df = schools_df[["school_name", "type", "size","budget"]]
school_summary_df.head()
school_summary_df["Per Student Budget"] = school_summary_df["budget"]/school_summary_df["size"]

In [156]:
# student_by_school_average = student_df["reading_score"].mean()
group_schools = student_df.groupby("school_name")

# Get the average of each column within the GroupBy object
schools_average_df = group_schools[["reading_score", "math_score"]].mean()

In [159]:
passing_read_count = student_df[student_df['reading_score'] >= 70].groupby("school_name")['reading_score'].count().reset_index()
passing_math_count = student_df[student_df['math_score'] >= 70].groupby("school_name")['math_score'].count().reset_index()
passing_scores = pd.merge(passing_read_count,passing_math_count,how="outer",on="school_name")


In [160]:
passing_overall = student_df[(student_df['math_score'] >= 70) & (student_df['reading_score'] >=70)].groupby("school_name")['student_name'].count().reset_index()
passing_scores_overall = pd.merge(passing_scores,passing_overall,how="outer",on="school_name")


In [161]:
combined_data = pd.merge(school_summary_df,schools_average_df,how="outer",on="school_name")
combined_all_data = pd.merge(combined_data,passing_scores_overall,how="outer",on="school_name")

In [162]:
combined_all_data["% Passing Math"] = combined_all_data["math_score_y"]/combined_all_data["size"]
combined_all_data["% Passing Reading"] = combined_all_data["reading_score_y"]/combined_all_data["size"]
combined_all_data["% Passing Overall"] = combined_all_data["student_name"]/combined_all_data["size"]

In [163]:
renamed_data = combined_all_data.rename(
    columns={"school_name": "School Name",
            "type": "School Type",
            "size":"School Size",
            "budget":"School Budget",
            "reading_score_x":"Average Reading Score",
            "math_score_x":"Average Math Score"
            })

final_school_summary = renamed_data[[
    "School Name",
    "School Type",
    "School Size",
    "School Budget",
    "Per Student Budget",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Passing Overall"
]]

final_school_summary2 = final_school_summary.copy()
final_school_summary2["School Budget"] = final_school_summary2["School Budget"].map("${:,.0f}".format)
final_school_summary2["Per Student Budget"] = final_school_summary2["Per Student Budget"].map("${:,.0f}".format)
final_school_summary2["Average Reading Score"] = final_school_summary2["Average Reading Score"].map("{:,.2f}".format)
final_school_summary2["Average Math Score"] = final_school_summary2["Average Math Score"].map("{:,.2f}".format)
final_school_summary2["% Passing Math"] = final_school_summary2["% Passing Math"].map("{:.2%}".format)
final_school_summary2["% Passing Reading"] = final_school_summary2["% Passing Reading"].map("{:.2%}".format)
final_school_summary2["% Passing Overall"] = final_school_summary2["% Passing Overall"].map("{:.2%}".format)
organize_alphabetically = final_school_summary2.sort_values(by="School Name",ascending=True)
organize_alphabetically

,School Name,School Type,School Size,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
7,Bailey High School,District,4976,"$3,124,928",$628,77.05,81.03,66.68%,81.93%,54.64%
6,Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
1,Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
13,Ford High School,District,2739,"$1,763,916",$644,77.10,80.75,68.31%,79.30%,54.29%
4,Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
3,Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
8,Holden High School,Charter,427,"$248,087",$581,83.80,83.81,92.51%,96.25%,89.23%
0,Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
12,Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%
9,Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


In [164]:
# Top 5 peforming schools by % passsing overall

top_performers = final_school_summary2.sort_values(by='% Passing Overall', ascending=False)
top_performers.head()

,School Name,School Type,School Size,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
6,Cabrera High School,Charter,1858,"$1,081,356",$582,83.06,83.98,94.13%,97.04%,91.33%
14,Thomas High School,Charter,1635,"$1,043,130",$638,83.42,83.85,93.27%,97.31%,90.95%
4,Griffin High School,Charter,1468,"$917,500",$625,83.35,83.82,93.39%,97.14%,90.60%
5,Wilson High School,Charter,2283,"$1,319,574",$578,83.27,83.99,93.87%,96.54%,90.58%
9,Pena High School,Charter,962,"$585,858",$609,83.84,84.04,94.59%,95.95%,90.54%


In [165]:
# Worst 5 peforming schools by % passsing overall

worst_performers = final_school_summary2.sort_values(by='% Passing Overall', ascending=True)
worst_performers.head()

,School Name,School Type,School Size,School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
11,Rodriguez High School,District,3999,"$2,547,363",$637,76.84,80.74,66.37%,80.22%,52.99%
1,Figueroa High School,District,2949,"$1,884,411",$639,76.71,81.16,65.99%,80.74%,53.20%
0,Huang High School,District,2917,"$1,910,635",$655,76.63,81.18,65.68%,81.32%,53.51%
3,Hernandez High School,District,4635,"$3,022,020",$652,77.29,80.93,66.75%,80.86%,53.53%
12,Johnson High School,District,4761,"$3,094,650",$650,77.07,80.97,66.06%,81.22%,53.54%


In [166]:
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

grade_9_math = student_df[student_df['grade'] == "9th"].groupby("school_name")['math_score'].mean().reset_index()
grade_10_math = student_df[student_df['grade'] == "10th"].groupby("school_name")['math_score'].mean().reset_index()
grade_11_math = student_df[student_df['grade'] == "11th"].groupby("school_name")['math_score'].mean().reset_index()
grade_12_math = student_df[student_df['grade'] == "12th"].groupby("school_name")['math_score'].mean().reset_index()
merge_9_10_grades = pd.merge(grade_9_math,grade_10_math, how="outer",on="school_name")
merge_11_12_grades=pd.merge(grade_11_math,grade_12_math,how="outer",on="school_name")
merge_all_grades = pd.merge(merge_9_10_grades,merge_11_12_grades,how="outer",on="school_name")
rename_grades_df = merge_all_grades.rename(columns={
    "school_name":"School Name",
    "math_score_x_x": "9th Grade",
    "math_score_y_x":"10th Grade",
    "math_score_x_y":"11th Grade",
    "math_score_y_y":"12th Grade"
})
rename_grades_df["9th Grade"] = rename_grades_df["9th Grade"].map("{:,.2f}".format)
rename_grades_df["10th Grade"] = rename_grades_df["10th Grade"].map("{:,.2f}".format)
rename_grades_df["11th Grade"] = rename_grades_df["11th Grade"].map("{:,.2f}".format)
rename_grades_df["12th Grade"] = rename_grades_df["12th Grade"].map("{:,.2f}".format)
rename_grades_df.style.set_caption('Average Math Scores by Grade')

,School Name,9th Grade,10th Grade,11th Grade,12th Grade
0,Bailey High School,77.08,77.00,77.52,76.49
1,Cabrera High School,83.09,83.15,82.77,83.28
2,Figueroa High School,76.40,76.54,76.88,77.15
3,Ford High School,77.36,77.67,76.92,76.18
4,Griffin High School,82.04,84.23,83.84,83.36
5,Hernandez High School,77.44,77.34,77.14,77.19
6,Holden High School,83.79,83.43,85.00,82.86
7,Huang High School,77.03,75.91,76.45,77.23
8,Johnson High School,77.19,76.69,77.49,76.86
9,Pena High School,83.63,83.37,84.33,84.12


In [167]:
# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

grade_9_reading = student_df[student_df['grade'] == "9th"].groupby("school_name")['reading_score'].mean().reset_index()
grade_10_reading = student_df[student_df['grade'] == "10th"].groupby("school_name")['reading_score'].mean().reset_index()
grade_11_reading = student_df[student_df['grade'] == "11th"].groupby("school_name")['reading_score'].mean().reset_index()
grade_12_reading = student_df[student_df['grade'] == "12th"].groupby("school_name")['reading_score'].mean().reset_index()
merge_9_10_reading_grades = pd.merge(grade_9_reading,grade_10_reading,how="outer",on="school_name")
merge_11_12_reading_grades = pd.merge(grade_11_reading,grade_12_reading,how="outer",on="school_name")
merge_all_reading_grades = pd.merge(merge_9_10_reading_grades,merge_11_12_reading_grades,how="outer",on="school_name")
rename_grades_reading_df = merge_all_reading_grades.rename(columns={
    "school_name":"School Name",
    "reading_score_x_x": "9th Grade",
    "reading_score_y_x":"10th Grade",
    "reading_score_x_y":"11th Grade",
    "reading_score_y_y":"12th Grade"
})
rename_grades_reading_df["9th Grade"] = rename_grades_reading_df["9th Grade"].map("{:,.2f}".format)
rename_grades_reading_df["10th Grade"] = rename_grades_reading_df["10th Grade"].map("{:,.2f}".format)
rename_grades_reading_df["11th Grade"] = rename_grades_reading_df["11th Grade"].map("{:,.2f}".format)
rename_grades_reading_df["12th Grade"] = rename_grades_reading_df["12th Grade"].map("{:,.2f}".format)
rename_grades_reading_df.style.set_caption('Average Reading Scores by Grade')


,School Name,9th Grade,10th Grade,11th Grade,12th Grade
0,Bailey High School,81.30,80.91,80.95,80.91
1,Cabrera High School,83.68,84.25,83.79,84.29
2,Figueroa High School,81.20,81.41,80.64,81.38
3,Ford High School,80.63,81.26,80.40,80.66
4,Griffin High School,83.37,83.71,84.29,84.01
5,Hernandez High School,80.87,80.66,81.40,80.86
6,Holden High School,83.68,83.32,83.82,84.70
7,Huang High School,81.29,81.51,81.42,80.31
8,Johnson High School,81.26,80.77,80.62,81.23
9,Pena High School,83.81,83.61,84.34,84.59


In [168]:
# Scores by School Spending

# Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
bins = [0, 584, 629, 644, 675]

# Create labels for these bins
group_labels = [
    "\$0 to \$584",
    "\$585 to \$629",
    "\$630 to \$644",
    "\$644 to \$675"
]

final_school_summary3 = final_school_summary.copy()
final_school_summary3["Average Spending Ranges"] = pd.cut(final_school_summary3["Per Student Budget"], bins, labels=group_labels)

# Create a GroupBy object based upon "View Group"
groupby = final_school_summary3.groupby("Average Spending Ranges")

# Get the average of each column within the GroupBy object
sorting = groupby[["Average Math Score", "Average Reading Score", "% Passing Math","% Passing Reading","% Passing Overall"]].mean()

formatting = sorting.copy()
formatting["Average Reading Score"] = formatting["Average Reading Score"].map("{:,.2f}".format)
formatting["Average Math Score"] = formatting["Average Math Score"].map("{:,.2f}".format)
formatting["% Passing Math"] = formatting["% Passing Math"].map("{:.2%}".format)
formatting["% Passing Reading"] = formatting["% Passing Reading"].map("{:.2%}".format)
formatting["% Passing Overall"] = formatting["% Passing Overall"].map("{:.2%}".format)
formatting.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Average Spending Ranges,,,,,
\$0 to \$584,83.46,83.93,93.46%,96.61%,90.37%
\$585 to \$629,81.90,83.16,87.13%,92.72%,81.42%
\$630 to \$644,78.52,81.62,73.48%,84.39%,62.86%
\$644 to \$675,77.00,81.03,66.16%,81.13%,53.53%


In [169]:
# Scores by School Size

# Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).

school_size = [0, 999, 1999, 5000]

# Create labels for these bins
size_labels = [
    "Small (< 1000)",
    "Medium (1000 to 1999)",
    "Large (2000 to 3500)"
]

final_school_summary4 = final_school_summary.copy()
final_school_summary4["School Size"] = pd.cut(final_school_summary3["School Size"], school_size, labels=size_labels)

# Create a GroupBy object based upon "View Group"
groupby_size = final_school_summary4.groupby("School Size")

# Get the average of each column within the GroupBy object
sorting_size = groupby_size[["Average Math Score", "Average Reading Score", "% Passing Math","% Passing Reading","% Passing Overall"]].mean()

formatting1 = sorting_size.copy()
formatting1["Average Reading Score"] = formatting1["Average Reading Score"].map("{:,.2f}".format)
formatting1["Average Math Score"] = formatting1["Average Math Score"].map("{:,.2f}".format)
formatting1["% Passing Math"] = formatting1["% Passing Math"].map("{:.2%}".format)
formatting1["% Passing Reading"] = formatting1["% Passing Reading"].map("{:.2%}".format)
formatting1["% Passing Overall"] = formatting1["% Passing Overall"].map("{:.2%}".format)
formatting1.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (< 1000),83.82,83.93,93.55%,96.10%,89.88%
Medium (1000 to 1999),83.37,83.86,93.60%,96.79%,90.62%
Large (2000 to 3500),77.75,81.34,69.96%,82.77%,58.29%


In [170]:
# Scores by School Type

final_school_summary5 = final_school_summary.copy()

groupby_type = final_school_summary5.groupby("School Type")

sorting_type = groupby_type[["Average Math Score", "Average Reading Score", "% Passing Math","% Passing Reading","% Passing Overall"]].mean()

formatting2 = sorting_type.copy()
formatting2["Average Reading Score"] = formatting2["Average Reading Score"].map("{:,.2f}".format)
formatting2["Average Math Score"] = formatting2["Average Math Score"].map("{:,.2f}".format)
formatting2["% Passing Math"] = formatting2["% Passing Math"].map("{:.2%}".format)
formatting2["% Passing Reading"] = formatting2["% Passing Reading"].map("{:.2%}".format)
formatting2["% Passing Overall"] = formatting2["% Passing Overall"].map("{:.2%}".format)
formatting2.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,90.43%
District,76.96,80.97,66.55%,80.80%,53.67%


In [171]:
# You must include a written description of at least two observable trends based on the data.

print("Observation 1: Based on the dataset, we can see that on average, students who attend Charter schools are more likely to perform better than District schools on reading, math and overall.")
print("-"*64)
print("Observation 2: From the dataset, we can also see that schools that spend fewer dollars per student, are more \nlikely to see higher performance from their students on reading, math and overall scores.")

Observation 1: Based on the dataset, we can see that on average, students who attend Charter schools are more likely to perform better than District schools on reading, math and overall.
----------------------------------------------------------------
Observation 2: From the dataset, we can also see that schools that spend fewer dollars per student, are more 
likely to see higher performance from their students on reading, math and overall scores.
